In [1]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, GRPO_CONNECTIONS_CONFIG

cfg = GRPO_CONNECTIONS_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
# cfg.model_id_or_path = SMOL_LM_135M
cfg.notebook_mode = True
cfg.max_samples = 1000

/home/brianf/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-29 08:34:26,664	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()

2025-03-29 08:34:45.252 | INFO     | trl_wrapper.trainer_wrapper:__init__:265 - Setting padding side to: left
2025-03-29 08:34:45.386 | INFO     | trl_wrapper.trainer_wrapper:init_model:276 - Using device: cuda:0
2025-03-29 08:34:45.387 | INFO     | trl_wrapper.trainer_wrapper:init_model:290 - Loading model Qwen/Qwen2.5-1.5B-Instruct with attn_impl: flash_attention_2


In [3]:
wrapper.init_data_module()

2025-03-29 08:36:42.319 | INFO     | trl_wrapper.wrapper_config:__init__:133 - Cache dir: ../dataset_caches/connections_data_module
Map: 100%|██████████| 353/353 [00:00<00:00, 5290.22 examples/s]


In [5]:
wrapper.data_module.train_dataset[0]

{'words': ['nba',
  'chest',
  'hoop',
  'band',
  'limit',
  'cap',
  'circle',
  'pearl',
  'curb',
  'traffic',
  'basket',
  'check',
  'paper',
  'bin',
  'hamper',
  'ring'],
 'solution': {'groups': [{'reason': 'circular shapes',
    'words': ['band', 'circle', 'hoop', 'ring']},
   {'reason': 'containers', 'words': ['basket', 'bin', 'chest', 'hamper']},
   {'reason': 'restrict', 'words': ['cap', 'check', 'curb', 'limit']},
   {'reason': '___ jam', 'words': ['nba', 'paper', 'pearl', 'traffic']}]},
 'prompt': [{'content': '\nYou are an expert puzzle solving model.\nFind groups of words that are related to each other. Each group is four words long.\nYou may only use each word in one group.\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n<group>\n...\n</group>\n<group>\n...\n</group>\n</answer>\n\n# Example\n\nUser: apple, orange, banana, pear, corolla, charger,\nAssistant: <reasoning>\nThe first group are all fruits.\nThe second group are all cars.\n</re

In [ ]:
wrapper.init_trainer()

In [ ]:
from model.reasoning import GSM8K_SYSTEM_PROMPT, CONNECTIONS_PROMPT
from model.utils import get_available_device

messages = [
    {"role": "system", "content": CONNECTIONS_PROMPT},
    {
        "role": "user",
        "content": "train, panda, dove, series, wind, bear, orca, bass, string, skunk, speed, sand, zebra, tourist, desert, chain",
    },
]
tokenized_chat = wrapper.tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
    return_attention_mask=True,
)

device = get_available_device()
tokenized_chat = tokenized_chat.to(device)
out = wrapper.model.generate(tokenized_chat, max_length=1024)
print(wrapper.tokenizer.decode(out[0], skip_special_tokens=True))

In [ ]:
wrapper.train()